<h1>Basic Demo</h1> 

Basic steps to process video for HRV estimation trough rPPG methods:
<ul>
    <li><a href="#video">Load video and crop faces</a></li>
    <li><a href="#roi">Process ROIs</a></li>
    <li><a href="#method">Apply rPPG methods</a></li>
    <li><a href="#res">Analyze results</a></li>
    <li><a href="#tune">Do fine tuning</a></li>
    <li><a href="#gt">Compare with ground truth (GT)</a></li>
</ul>

In [1]:
# -- Modules and packages to import for demo
from pyVHR.signals.video import Video
from pyVHR.methods.pos import POS
from pyVHR.methods.chrom import CHROM
from pyVHR.analysis.testsuite import TestSuite, TestResult


i


<a id="video"></a>
<h2>Load video and crop faces</h2> 

Given a video sequence $v(t)$, the process starts by extracting the portion of the image corresponding to the face from each frame.
The face detectors included in the Package pyVHR are: dlib, mtcnn, and Kalman filter for face tracking. Thus,
\begin{equation*}
b(t)=
\begin{cases} 
    \text{dlib}(v(t))  \\ 
    \text{mtcnn}(v(t)) \\ 
    \text{kalman}(v(t)).
\end{cases}
\end{equation*}
The signal $b(t)$ has dimensions $w\times h \times 3 \times W_s$, where $w$ and $h$ are the width and the height of the bounding box containing the face, respectively. Signal $b(t)$ has 3 channels being coded in the RGB-color space, and depth $W_s$ according to the time window considered.

<h3>Download video sample</h3> 
To repeat next experiment:
<ul>
    <li>Download the file <code>ID1.zip</code> (7.57GB) from <a href="https://github.com/partofthestars/LGI-PPGI-DB">github.com/partofthestars/LGI-PPGI-DB</a></li>
    <li>Unpack the file in the pyVHR root directory and rename the folder as <code>sampleDataset</code></li>
    <li>Run the following snippets (cells)</li>
        

In [2]:
# -- Video object
videoFilename = ("face","mp4")
video = Video(videoFilename)

# -- extract faces
video.getCroppedFaces(detector='mtcnn', extractor='skvideo')

video.printVideoInfo()

print("\nShow video cropped faces, crop size:", video.cropSize)
video.showVideo()


   * Video filename: ../sampleDataset/face.mp4
         Total frames: 301
             Duration: 10.03 (sec)
           Frame rate: 30 (fps)
                Codec: h264
           Num frames: 301
               Height: 592
                Width: 592
             Detector: mtcnn
            Extractor: skvideo

Show video cropped faces, crop size: (476, 356, 3)


interactive(children=(IntSlider(value=1, description='frame', max=301), Output()), _dom_classes=('widget-inter…

<a id="roi"></a>
<h2>Process ROIs</h2> 

For each frame $t=1,\dots,T$, given the ROIs, either rectangular patch-based $R(t)$ or skin-based $S(t)$, we average over all the selected pixels to compute the output  $q(t)$ of this step. More formally, if $N$ denotes the number of rectangular patches, $\left|R^{(i)}(t)\right|$ the number of pixels in the $i$-th patch, and  $\left|S(t) \right |$ the number of detected skin pixels within the face, we have
\begin{equation*}
q(t)=\begin{cases} 
    \text{Patch}(R(t)) \\ \text{Skin}(S(t))
    \end{cases}
\end{equation*}
where
\begin{align*}
    \text{Patch}(R(t))&= 
    \frac{1}{N}\sum_{i=1}^N \frac{1}{\left| R^{(i)}(t) \right|} \sum_{(x,y)\in R^{(i)}(t)} R^{(i)}_{x,y}(t)   \\ 
    \text{Skin}(S(t)) &= 
    \frac{1}{\left| S(t) \right |} \sum_{(x,y)\in S(t)}
    S_{x,y}(t)
\end{align*}


In [3]:
# -- define ROIs: free rectangular regions
video.setMask(typeROI='rect', rectCoords=[[15,20,140,50],[10,120,100,30]])
video.printROIInfo()
video.showVideo()

      ROI type: rect
   Rect coords: [[15, 20, 140, 50], [10, 120, 100, 30]]


interactive(children=(IntSlider(value=1, description='frame', max=301), Output()), _dom_classes=('widget-inter…

In [4]:
# -- define ROIs: standard regions, i.e. 'forehead', 'lcheek', 'rcheek', 'nose'
video.setMask(typeROI='rect', rectRegions=['forehead', 'lcheek', 'rcheek', 'nose'])
video.printROIInfo()
video.showVideo()

      ROI type: rect
   Rect coords: [[71, 47, 213, 57], [53, 257, 53, 52], [249, 257, 53, 52], [124, 238, 106, 38]]


interactive(children=(IntSlider(value=1, description='frame', max=301), Output()), _dom_classes=('widget-inter…

In [12]:
# -- define ROIs: using skin, with threshold param 
video.setMask(typeROI='skin_adapt',skinThresh_adapt=0.2)
video.printROIInfo()
video.showVideo()

      ROI type: skin_adapt
   Skin thresh: 0.2


interactive(children=(IntSlider(value=1, description='frame', max=301), Output()), _dom_classes=('widget-inter…

In [41]:
# -- define ROIs: using skin, with threshold param 
video.setMask(typeROI='skin_fix',skinThresh_fix=[10,100])
video.printROIInfo()
video.showVideo()

      ROI type: skin_fix
   Skin thresh: [10, 100]


interactive(children=(IntSlider(value=1, description='frame', max=301), Output()), _dom_classes=('widget-inter…

<a id="method"></a>
<h2>Apply rPPG methods</h2> 

The temporal RGB trace $x(t) =(x_r(t),x_g(t),x_b(t))^T$ is achieved by preprocessing the raw RGB signal $s(t) =(s_r(t),s_g(t),s_b(t))^T$. The signal $x(t)$ is than split into overlapped subsequences, each  representing samples of a finite-length multivariate measurement with $t= 1,2,...,M$, where $M=W_s\cdot f_{ps}$ is the number of frames selected by the sliding window, being $f_{ps}$ the video frame rate. Thus, a method receives as input a chunk of the sequence $x(t)$ and produces as output a monovariate temporal sequence $y(t)$, i.e. a real BVP estimate coming from the application of the rPPG model.

In [7]:
# -- Define a configuration file. 
#    It contains all the information relative to the dataset (e.g. the path), 
#    and the test procedure (e.g. hyperparamenters)
cfgFilename = '../pyVHR/analysis/sample.cfg'

# -- apply the pipeline until GT comparison
test = TestSuite(configFilename=cfgFilename)

# -- run exp and save results on a pandas file
#    change verb to see more details as follow:
#       0 - not verbose
#       1 - show the main steps
#       2 - display graphic 
#       3 - display spectra  
#       4 - display errors
#       (use also combinations, e.g. verb=21, verb=321)

result = test.start(outFilename='sampleExp.h5', verb='1')

** Run the test with the following config:
      dataset: LGI_PPGI
      methods: ['POS', 'CHROM']
cms50
cms50
cms50
cms50
cms50
cms50
cms50
cms50
cms50
cms50
cms50
cms50
cms50


<a id="test"></a>
<h2>Test results</h2>

Results from experiments are organized in pandas file and object of type TestResult. 

In [8]:
# -- pandas file for collecting results
result.dataFrame

""


<a id="tune"></a>
<h2>Do fine tuning</h2>

In this section it is shown how to tune parameters, visualize intermediate results, invoke methods on specific dataset 

In [19]:
# -- Detailed pipeline for rafinement steps and tuning  

# -- define some params in the form of dict (those in the cfg file) 
params = {"video": video, "verb":2, "ROImask":"rect","rectRegions":['nose'], "skinAdapt":0.2,"csv_filename":"mesh"+".csv"}

# -- invoke the method
m = CHROM(**params)
#m = POS(**params)

# -- invoke the method
bpmES, timesES = m.runOffline(**params)

[-2.87723382e-02  1.73875646e-01  3.50982763e-01  4.73844571e-01
  5.16118786e-01  4.66915984e-01  3.43238412e-01  1.90158742e-01
  6.37784990e-02  4.28738150e-03  1.54026378e-02  6.46871770e-02
  1.06677465e-01  1.15245180e-01  1.03770872e-01  1.17722383e-01
  2.01449526e-01  3.58992007e-01  5.35411773e-01  6.34814085e-01
  5.68899600e-01  3.09348585e-01 -8.75650117e-02 -5.00695464e-01
 -7.98621724e-01 -8.99198048e-01 -8.02107981e-01 -5.79373238e-01
 -3.33043894e-01 -1.45810607e-01 -5.03282217e-02 -2.89257287e-02
 -3.73744244e-02 -3.52633770e-02 -6.08362243e-03  4.04828459e-02
  8.05410865e-02  9.24262210e-02  6.83611611e-02  1.71327373e-02
 -4.09606609e-02 -8.27280341e-02 -9.01825920e-02 -5.57108026e-02
  1.63922037e-02  1.11726574e-01  2.10163210e-01  2.91648664e-01
  3.41858886e-01  3.55822055e-01  3.37908419e-01  2.97843009e-01
  2.44511986e-01  1.81210423e-01  1.05910665e-01  1.72155511e-02
 -7.77839311e-02 -1.60236270e-01 -2.05447044e-01 -1.96403999e-01
 -1.36864215e-01 -5.43620

<a id="gt"></a>
<h2>Compare with ground truth (GT)</h2>

In this section it is shown how to define a new Dataset object of type SAMPLE, to load the BVP ground truth (GT) signal and plot it and to compute the GT BPMs used in comparisons with those estimated by method

In [15]:
from pyVHR.datasets.sample import SAMPLE
from pyVHR.datasets.dataset import Dataset

# -- dataset object
dataset = SAMPLE(videodataDIR='../sampleDataset/', BVPdataDIR='../sampleDataset/')

# -- ground-truth (GT) signal
idx = 0   # index of signal within the list dataset.videoFilenames
fname = dataset.getSigFilename(idx)

# -- load signal and build a BVPsignal or ECGsignal object
sigGT = dataset.readSigfile(fname)
sigGT.plot()

# -- plot signal + peaks
sigGT.findPeaks(distance=20)
sigGT.plotBPMPeaks()

# -- compute BPM GT
winSizeGT = 7
bpmGT, timesGT = sigGT.getBPM(winSizeGT)

cms50
cms50
cms50
cms50
cms50
cms50
cms50
cms50
cms50
cms50
cms50
cms50
cms50


IndexError: list index out of range

In [ ]:
from pyVHR.utils.errors import getErrors, printErrors, displayErrors

# -- error metrics
RMSE, MAE, MAX, PCC = getErrors(bpmES, bpmGT, timesES, timesGT)
printErrors(RMSE, MAE, MAX, PCC)
displayErrors(bpmES, bpmGT, timesES, timesGT)

In [ ]:
# -- print BPM
print("BPMs of the GT signal averaged on winSizeGT = %d sec:" %winSizeGT)
print(bpmGT)

# -- plot spectrogram
sigGT.displaySpectrum()